In [1]:
import tensorflow as tf
import numpy as np
from sklearn.kernel_approximation import RBFSampler
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten



In [2]:

# 加载MNIST数据集
mnist = fetch_openml('mnist_784')
X = mnist.data.astype('float64')
y = mnist.target.astype('int64')
X /= 255.0

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


d:\anaconda\envs\tensorflow\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:

# 初始化RFF特征映射
gamma = 1.0
n_components = 1000
rff = RBFSampler(gamma=gamma, n_components=n_components, random_state=42)
X_train_rff = rff.fit_transform(X_train)
X_test_rff = rff.transform(X_test)

# 标准化RFF特征
scaler = StandardScaler()
X_train_rff = scaler.fit_transform(X_train_rff)
X_test_rff = scaler.transform(X_test_rff)

# 将数据转换为NumPy数组
X_train_rff = np.array(X_train_rff, dtype=np.float32)
X_test_rff = np.array(X_test_rff, dtype=np.float32)
y_train = np.array(y_train, dtype=np.int64)
y_test = np.array(y_test, dtype=np.int64)


In [4]:
print(X_train_rff.shape)

(56000, 1000)


In [5]:

# 定义神经网络模型
# class NeuralNetwork(tf.keras.Model):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(NeuralNetwork, self).__init__()
#         self.hidden = tf.keras.layers.Dense(hidden_dim, activation='relu')
#         self.output_layer = tf.keras.layers.Dense(output_dim)
#         self.softmax = tf.keras.layers.Dense(output_dim, activation='softmax')

#     def call(self, x):
#         x = self.hidden(x)
#         x = self.output_layer(x)
#         x = self.softmax(x)
#         x = np.argmax(x)
#         return x

def creat_model(hidden_dim, output_dim):
    model = Sequential([
        # Flatten(input_shape=[, input_dim]),
        Dense(hidden_dim, activation='relu'),
        Dense(output_dim, activation='softmax')
    ])

    return model
    
# 初始化模型
input_dim = n_components
hidden_dim = 100
output_dim = 10

model = creat_model(hidden_dim, output_dim)

# 定义损失函数和优化器
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# 编译模型
model.compile(optimizer=optimizer, loss=loss_fn , metrics=['accuracy'])

# 训练模型
history = model.fit(X_train_rff, y_train, epochs=10, batch_size=32, verbose=1)



# 在测试集上评估模型
# test_loss, test_accuracy = model.evaluate(X_test_rff, y_test, verbose=0)
# print(f'Accuracy on test set: {test_accuracy*100:.2f}%')


Epoch 1/10


d:\anaconda\envs\tensorflow\lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1750/1750 [==============================] - 2s 1ms/step - loss: 2.4304 - accuracy: 0.1039
Epoch 2/10
1750/1750 [==============================] - 2s 1ms/step - loss: 2.2344 - accuracy: 0.1778
Epoch 3/10
1750/1750 [==============================] - 2s 1ms/step - loss: 2.1591 - accuracy: 0.2183
Epoch 4/10
1750/1750 [==============================] - 2s 1ms/step - loss: 2.0344 - accuracy: 0.2807
Epoch 5/10
1750/1750 [==============================] - 2s 1ms/step - loss: 1.8766 - accuracy: 0.3432
Epoch 6/10
1750/1750 [==============================] - 2s 1ms/step - loss: 1.7272 - accuracy: 0.4018
Epoch 7/10
1750/1750 [==============================] - 2s 1ms/step - loss: 1.6018 - accuracy: 0.4498
Epoch 8/10
1750/1750 [==============================] - 2s 1ms/step - loss: 1.5008 - accuracy: 0.4847
Epoch 9/10
1750/1750 [==============================] - 2s 1ms/step - loss: 1.4167 - accuracy: 0.5143
Epoch 10/10
1750/1750 [==============================] - 2s 1ms/step - loss: 1.3464 - accurac

In [6]:
pred = np.argmax(model.predict(X_test_rff), axis=1)
true_value = 0
for i in range(pred.shape[0]):
    if pred[i] == y_test[i]:
        true_value += 1
accuracy = true_value / pred.shape[0]
print(accuracy)

438/438 [==============================] - 0s 869us/step


0.10678571428571429
